### Example: Steam table interpolation

We use a small saturated steam table as an illustrative example.
The columns are pressure (MPa), saturation temperature ($\degree C$), saturated liquid enthalpy $h_f$ (kJ/kg), saturated vapor enthalpy $h_g$ (kJ/kg), saturated liquid specific volume $v_f$ (m^3/kg), saturated vapor specific volume $v_g$ (m^3/kg).

In [ ]:
# Columns: pressure (MPa), Tsat (C), hf (kJ/kg), hg (kJ/kg), vf (m^3/kg), vg (m^3/kg)
saturated_table = [
    # P,   Tsat,    hf,     hg,      vf,      vg
    (0.01, 45.81, 191.8, 2583.9, 0.00101, 14.67),
    (0.05, 81.33, 340.5, 2675.6, 0.00102, 3.459),
    (0.1, 99.61, 417.4, 2675.5, 0.00104, 1.694),
    (0.2, 120.23, 504.7, 2704.7, 0.00106, 0.8857),
    (0.5, 151.83, 640.1, 2748.0, 0.00109, 0.3749),
    (1.0, 179.88, 762.5, 2778.1, 0.001, 0.1944),
]

pressures = [row[0] for row in saturated_table]
Tsat_list = [row[1] for row in saturated_table]
hf_list = [row[2] for row in saturated_table]
hg_list = [row[3] for row in saturated_table]

We use linear interpolation in this steam table example.

In [ ]:
from typing import List, Tuple

def linear_interpolate(x0: float, y0: float, x1: float, y1: float, x: float) -> float:
    """Linear interpolation (assumes x0 != x1)."""
    if x1 == x0:
        raise ValueError("x0 and x1 must be different for linear interpolation")
    t = (x - x0) / (x1 - x0)
    return y0 + t * (y1 - y0)


def find_bracketing_indices(x_list: List[float], x: float) -> Tuple[int, int]:
    """Find indices i, i+1 in sorted x_list such that x_list[i] <= x <= x_list[i+1].
    If x is outside range, returns the nearest two indices for extrapolation.
    """
    if x_list[0] >= x:
        return 0, 1
    if x_list[-1] <= x:
        return len(x_list) - 2, len(x_list) - 1
    for i in range(len(x_list) - 1):
        if x_list[i] <= x <= x_list[i + 1]:
            return i, i + 1
    return 0, 1

def interpolate_1d_table(x_list: List[float], y_list: List[float], x: float) -> float:
    """Given monotonic x_list and corresponding y_list, interpolate to x."""
    i0, i1 = find_bracketing_indices(x_list, x)
    return linear_interpolate(x_list[i0], y_list[i0], x_list[i1], y_list[i1], x)


#### Example 1
Interpolate saturated vapor enthalpy $h_g$ at an intermediate pressure.

In [ ]:
def hg_at_pressure(p_MPa: float) -> float:
    """Interpolate saturated vapor enthalpy hg at given pressure p (MPa)."""
    return interpolate_1d_table(pressures, hg_list, p_MPa)

In [ ]:
# 1D interpolation example
p_test = 0.3
print(
f"""
Interpolated saturated vapor enthalpy hg at P={p_test} MPa:
{hg_at_pressure(p_test):.2f} kJ/kg"""
)

#### Example 2
Compute the specific enthalpy at a given pressure and steam quality (x).


Remember:
$h=(1-x)\cdot h_f+x \cdot h_g$

In [ ]:
def enthalpy_from_pressure_and_quality(p_MPa: float, quality: float) -> float:
    """Return specific enthalpy (kJ/kg) for saturated mixture at pressure P and vapor quality x in [0,1]."""
    if not (0.0 <= quality <= 1.0):
        raise ValueError("quality must be between 0 and 1")
    hf = interpolate_1d_table(pressures, hf_list, p_MPa)
    hg = interpolate_1d_table(pressures, hg_list, p_MPa)
    return (1.0 - quality) * hf + quality * hg


In [ ]:
# enthalpy of mixture: P=0.3 MPa and quality x=0.2
x_test = 0.2
h = enthalpy_from_pressure_and_quality(p_test, x_test)
print(f"Specific enthalpy at P={p_test} MPa and quality x={x_test}: {h:.2f} kJ/kg")

In [ ]:
# Plot the results
import matplotlib.pyplot as plt
import numpy as np
p_values = np.linspace(0.01, 1.0, 100)
hg_values = [hg_at_pressure(p) for p in p_values]
# scatter the original data points
plt.scatter(pressures, hg_list, color='red', label='Data Points')
# plot the interpolated curve
plt.plot(p_values, hg_values)
plt.xlabel("Pressure (MPa)")
plt.ylabel("Saturated Vapor Enthalpy hg (kJ/kg)")
plt.title("Saturated Vapor Enthalpy vs Pressure")
plt.grid()
plt.show()